In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import Adam

In [4]:
np.set_printoptions(suppress=True)

# MULTIPLE ITEMS

## Generate data

In [5]:
n = 5000

In [6]:
prices = np.random.uniform(50, 200, size=(n, 1)).astype(np.float32)
xs = np.random.normal(size=(n,5)).astype(np.float32)

In [7]:
# create input for item ids, we'll make 5
ids = np.tile([0,1,2,3,4], n//5).reshape(-1,1)

In [8]:
# create beta coefficients
beta_0 = np.array([500],)

beta_p = np.array([-1.4,
                   -1.5,
                   -1.6,
                   -1.8,
                   -2.0],)

betas_x = np.random.normal(size=(5,1)).astype(np.float32)
# betas_x

In [9]:
# optimal prices
opt_prices = (-beta_0/(2*beta_p)).round(2)
opt_prices

array([178.57, 166.67, 156.25, 138.89, 125.  ])

In [10]:
# noise
noise = np.random.normal(size=(n,1)).astype(np.float32)

In [11]:
# create demand to forecast
y = beta_0 + beta_p[ids]*prices + np.dot(xs, betas_x) + noise
y = np.maximum(0, y)

In [12]:
# plt.hist(y)

In [13]:
trn = np.arange(int(n*.8))
tst = np.arange(int(n*.8), n)

## Model demand

In [14]:
# price and X inputs
inp_p = Input((1,), name='inp_p')
inp_x = Input((5,), name='inp_x')

In [15]:
# going to use embeddings to learn the "coefficient" for each item
inp_ids = Input((1,), name='inp_ids')
ids_emb = Embedding(input_dim=5, output_dim=1, name='ids_emb')(inp_ids)
ids_emb = Reshape((1,), name='ids_emb_re')(ids_emb)

# multiply the embedding coefficient by price
x = Multiply(name='mult_p_emb')([ids_emb, inp_p])

In [16]:
# concat the inputs
x = Concatenate(name='concat')([x, inp_x])

# single dense hidden layer
x = Dense(10, name='hid')(x)

# output for predicting demand
out = Dense(1, name='out')(x)

# build and compile the model
model = Model([inp_ids, inp_p, inp_x], out)
model.compile(Adam(lr=.01), loss='mean_squared_error')

In [17]:
# train
hist = model.fit([ids[trn], prices[trn], xs[trn]], y[trn],
                 validation_data=[[ids[tst], prices[tst], xs[tst]], y[tst]],
                 verbose=0,
                 epochs=50)

## Price optimization

In [18]:
# make model copy for price
model_p = keras.models.clone_model(model)
model_p.set_weights(model.get_weights())

In [19]:
# "freeze" the model
for layer in model_p.layers:
    layer.trainable = False

In [20]:
ls = {l.name:l for l in model_p.layers}
ls

{'concat': <keras.layers.merge.Concatenate at 0x10fb35048>,
 'hid': <keras.layers.core.Dense at 0x10fe71828>,
 'ids_emb': <keras.layers.embeddings.Embedding at 0x10fb35128>,
 'ids_emb_re': <keras.layers.core.Reshape at 0x10fb35f98>,
 'inp_ids': <keras.engine.topology.InputLayer at 0x10e6b49b0>,
 'inp_p': <keras.engine.topology.InputLayer at 0x10e6b4e80>,
 'inp_x': <keras.engine.topology.InputLayer at 0x10e6b4d30>,
 'mult_p_emb': <keras.layers.merge.Multiply at 0x10fb35be0>,
 'out': <keras.layers.core.Dense at 0x10fe86f60>}

In [21]:
# id price embeddings
# lets us "train" a price for every id
p = Embedding(input_dim=5, output_dim=1, name='ids_price')(ls['inp_ids'].output)
p = Reshape((1,), name='ids_price_re')(p)

In [22]:
x = ls['mult_p_emb']([p, ls['ids_emb_re'].output, ls['inp_p'].output])

In [23]:
# concat the inputs and send through hidden layers like before
x = ls['concat']([x, ls['inp_x'].output])
x = ls['hid'](x)
x = ls['out'](x)

In [24]:
# our output of the last model was an estimate for demand
# so now we multiply that by the price to get revenue
out = Multiply(name='revenue_out')([p, x])

In [25]:
# build model
model_p = Model([ls['inp_ids'].input, ls['inp_p'].input, ls['inp_x'].input], out)

In [26]:
# revenue loss
def rev_loss(y_true, y_pred):
    return -K.mean(y_pred)

In [27]:
# compile model
model_p.compile(Adam(lr=.1), loss=rev_loss)

In [28]:
# callback to track layer weights
class WtTracker(keras.callbacks.Callback):
    def __init__(self, layer, display=5):
        self.seen = 0
        self.layer = layer
        self.display = display

    def on_epoch_end(self,batch,logs={}):
        self.seen += 1
        if self.seen % self.display == 0:
            wts = self.layer.get_weights()[0].flatten().round(2)
            print(f'epoch {self.seen}, price {wts}')

In [29]:
# calculated optimal prices for reference
list(opt_prices)

[178.57, 166.67, 156.25, 138.89, 125.0]

In [30]:
# train
hist = model_p.fit([ids, np.ones((len(xs), 1)), xs], y,
                   epochs=50,
                   verbose=0,
                   callbacks=[WtTracker(model_p.get_layer('ids_price'))])

epoch 5, price [66.16 65.66 65.79 64.22 63.15]
epoch 10, price [117.3  114.62 113.2  107.59 102.57]
epoch 15, price [151.98 145.96 141.98 129.85 119.62]
epoch 20, price [170.19 160.78 154.13 136.82 123.47]
epoch 25, price [176.31 164.87 156.97 137.8  123.79]
epoch 30, price [177.33 165.38 157.24 137.85 123.81]
epoch 35, price [177.39 165.4  157.25 137.85 123.81]
epoch 40, price [177.38 165.4  157.26 137.85 123.82]
epoch 45, price [177.39 165.39 157.25 137.83 123.83]
epoch 50, price [177.38 165.39 157.25 137.84 123.8 ]
